In [1]:
import numpy as np

$$
\newcommand{\d}{{\mathrm{d}}}
\newcommand{\A}{{\mathbf{A}}}
\newcommand{\U}{{\mathbf{U}}}
\newcommand{\S}{{\mathbf{S}}}
\newcommand{\V}{{\mathbf{V}}}
\newcommand{\F}{{\mathbf{F}}}
\newcommand{\I}{{\mathbf{I}}}
\newcommand{\dA}{{\d\A}}
\newcommand{\dU}{{\d\U}}
\newcommand{\dS}{{\d\S}}
\newcommand{\dV}{{\d\V}}
\newcommand{\Ut}{{\U^{\top}}}
\newcommand{\Vt}{{\V^{\top}}}
\newcommand{\dAt}{{\dA^{\top}}}
\newcommand{\gA}{{\overline{\A}}}
\newcommand{\gU}{{\overline{\U}}}
\newcommand{\gUt}{{\gU^{\top}}}
\newcommand{\gS}{{\overline{\S}}}
\newcommand{\gSt}{{\gS^{\top}}}
\newcommand{\gV}{{\overline{\V}}}
\newcommand{\gVt}{{\gV^{\top}}}
$$

SVD's forward mode autodiff is defined by the formulae for the differentials $\dU$, $\dS$, and $\dV$ in terms of $\dA$, $\U$, $\S$, and $\V$

$$
\dU = \U ( \F \circ [\Ut \dA \V \S + \S \Vt \dAt \U] ) + (\I_m - \U \Ut ) \dA \V \S^{-1}
$$
$$
\dS = \I_k \circ [\Ut \dA \V]
$$
$$
\dV = \V (\F \circ [\S \Ut \dA \V + \Vt \dAt \U \S]) + (\I_n - \V \Vt) \dAt \U \S^{-1}
$$

where
$$
F_{ij} = \frac{1}{s_j^2 - s_i^s}, i \neq j \newline
F_{ij} = 0, i = j
$$

In [2]:
def svd_jvp(A, dA, *, full_matrices=True, compute_uv=True):
    pass

SVD's reverse mode autodiff is defined by the formula

$$
\newcommand{\gAa}{{[\U (\F \circ [\Ut \gU - \gUt \U]) \S + (\I_m - \U \Ut) \gU \S^{-1} ] \Vt}}
\newcommand{\gAb}{{\U (\I_k \circ \gS ) \Vt}}
\newcommand{\gAc}{{\U [\S (\F \circ [\Vt \gV - \gVt \V]) \Vt + \S^{-1} \gVt (\I_n - \V \Vt)]}}
\gA = \gAa \newline + \gAb \newline + \gAc
$$

In [3]:
def svd_vjp(A, U, S, Vh, gU, gS, gVh):
    pass

In [27]:
A = np.array([[1.,2.], [5., 2.], [10., 23.], [10., 23.]])

U, S, Vh = np.linalg.svd(A)

print(U)
print(S)
print(Vh)

[[-0.06249881 -0.0248252  -0.70550606 -0.70550606]
 [-0.10831214 -0.99361751  0.02227914  0.02227914]
 [-0.70155626  0.07780729  0.52970552 -0.47029448]
 [-0.70155626  0.07780729 -0.47029448  0.52970552]]
[35.74646973  3.76694863]
[[-0.40941623 -0.91234772]
 [-0.91234772  0.40941623]]


Sources:

SVD real differentiability:
- https://j-towns.github.io/papers/svd-derivative.pdf
- https://people.maths.ox.ac.uk/gilesm/files/NA-08-01.pdf
- https://arxiv.org/pdf/1509.07838.pdf

SVD complex differentiability:
- https://arxiv.org/pdf/1909.02659.pdf
- https://giggleliu.github.io/2019/04/02/einsumbp.html

Existing implementations:

Jax forward:
- https://github.com/google/jax/blob/2a00533e3e686c1c9d7dfe9ed2a3b19217cfe76f/jax/_src/lax/linalg.py#L1578
- Jax only implements the forward rule because jax can derive the backward rule from the forward rule and vice versa.

Pytorch forward:
- https://github.com/pytorch/pytorch/blob/7a8152530d490b30a56bb090e9a67397d20e16b1/torch/csrc/autograd/FunctionsManual.cpp#L3122

Pytorch backward:
- https://github.com/pytorch/pytorch/blob/7a8152530d490b30a56bb090e9a67397d20e16b1/torch/csrc/autograd/FunctionsManual.cpp#L3228

Tensorflow forward:
- https://github.com/tensorflow/tensorflow/blob/bbe41abdcb2f7e923489bfa21cfb546b6022f330/tensorflow/python/ops/linalg_grad.py#L815

General complex differentiability:
- https://mediatum.ub.tum.de/doc/631019/631019.pdf

In [35]:
import jax.numpy as jnp
import jax

def f(A, B):
    return jnp.trace(A @ B)

A = jnp.array([[1, 2], [3, 4]], dtype=float)
B = jnp.array([[1,2,3], [4,5,6]], dtype=float)

print(f(A, B))

jax.jacfwd(f, argnums=(0, 1))(A, B)

35.0


(DeviceArray([[1., 4.],
              [2., 5.]], dtype=float32),
 DeviceArray([[1., 3., 0.],
              [2., 4., 0.]], dtype=float32))

In [93]:
from jax import custom_jvp
import jax.numpy as jnp

# f :: a -> b
@custom_jvp
def f(x):
    return jnp.dot(x, x)

# f_jvp :: (a, T a) -> (b, T b)
def f_jvp(primals, tangents):
    x, = primals
    t, = tangents
    return f(x), 2 * x @ t

f.defjvp(f_jvp)

<function __main__.f_jvp(primals, tangents)>

In [94]:
jax.grad(f)(jnp.array([1.,2.,4.]))

DeviceArray([2., 4., 8.], dtype=float32)

In [95]:
jax.make_jaxpr(jax.vjp(f, jnp.array([1.,2.,3.]))[1])(1.)

{ lambda a:f32[3]; b:f32[]. let
    c:f32[3] = dot_general[
      dimension_numbers=(((), ()), ((), ()))
      precision=None
      preferred_element_type=None
    ] b a
  in (c,) }

In [96]:
jax.make_jaxpr(jax.vjp(lambda x: x @ x, jnp.array([1.,2.,3.]))[1])(1.)

{ lambda a:f32[3]; b:f32[]. let
    c:f32[3] = dot_general[
      dimension_numbers=(((), ()), ((), ()))
      precision=None
      preferred_element_type=None
    ] b a
    d:f32[3] = dot_general[
      dimension_numbers=(((), ()), ((), ()))
      precision=None
      preferred_element_type=None
    ] b a
    e:f32[3] = add_any d c
  in (e,) }

In [99]:
jax.jacrev(f)(jnp.array([1.,2.,3.]))

DeviceArray([2., 4., 6.], dtype=float32)

In [100]:
jax.jacrev(lambda x: x @ x)(jnp.array([1., 2., 3.]))

DeviceArray([2., 4., 6.], dtype=float32)

In [90]:
2 * jnp.array([1.,2.])

DeviceArray([2., 4.], dtype=float32)